## Extração da identificação das escolas que fazem parte do PEI

O Governo do Estado fornece os dados das escolas que fazem parte do PEI em seu portal.

Os dados podem ser acessados em: https://transparencia.educacao.sp.gov.br/Home/DetalhesPrograma?programa=1027

Nesse notebook, realizamos o webscrapping desses dados, emulando a requisição de pesquisa que a interface de usuário faz (requisição post).

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
URL_POST_ESCOLAS_PEI = 'https://transparencia.educacao.sp.gov.br/Home/conteudoTabelaPrograma'
URL_POST_ESCOLAS_PEI

'https://transparencia.educacao.sp.gov.br/Home/conteudoTabelaPrograma'

In [3]:
PAYLOAD_BUSCA = dict(
    tipo=1,
    #diretoria=,
    #escola=,
    #municipio=,
    #endereco=,
    programa=27 #id do programa PEI
)

In [4]:
def get_table_from_pei_website():
    """
    Fetches the PEI data from the website and returns its html.
    """
    with requests.post(URL_POST_ESCOLAS_PEI, data=PAYLOAD_BUSCA) as response:
        if response.status_code == 200:
            table = response.text
            return table
        else:
            print('Error:', response.status_code, response.text)


In [5]:
html_table = get_table_from_pei_website()
html_table[:100]

'\r\n\r\n<section class="container_maps">\r\n    <div class="maps" style="width:904px; height:500px" id="ma'

In [6]:
def get_table_element(html_table):
    """
    Parses the HTML table and returns the table element.
    """
    soup = BeautifulSoup(html_table, 'html.parser')
    table = soup.find('table')
    
    return table

In [7]:
table = get_table_element(html_table)

In [8]:
#colunas
table.find_all('tr')[0].find_all('th')

[<th>Escola</th>,
 <th>Diretoria de Ensino</th>,
 <th>Município</th>,
 <th>Ações</th>]

In [9]:
#primeira linha
table.find_all('tr')[1].find_all('td')

[<td>
 <a class="text-captalize" href="/Home/DetalhesEscola?codesc=926085" id="NomeEscola">bairro da bocaina</a>
 </td>,
 <td><span class="text-captalize" id="diretoriaEnsino">guaratingueta</span></td>,
 <td><span class="text-captalize" id="municipio">cunha</span></td>,
 <td class="icones-acoes">
 <a data-tooltip="Rota da Escola" href="https://www.google.com/maps/dir//-23.0361,-44.9253/@-23.0361,-44.9253,17z/data=!3m1!4b1" id="btnRota" target="_blank">
 <img alt="" src="/Contents/img/icones/icon-trajeto.svg"/>
 </a> 
                     <a data-tooltip="Informações da Escola" href="/Home/DetalhesEscola?codesc=926085" id="btnDetalhesEscola">
 <img alt="" src="/Contents/img/icones/icon-detalhes.svg"/>
 </a>
 </td>]

In [10]:
def get_colls(table):
    return [th.text.strip() for th in table.find_all('tr')[0].find_all('th')]

In [11]:
cols = get_colls(table)
cols

['Escola', 'Diretoria de Ensino', 'Município', 'Ações']

In [12]:
#ultima coluna - dados com links para parsear
exemplo_ultima_coluna = table.find_all('tr')[1].find_all('td')[-1]
exemplo_ultima_coluna

<td class="icones-acoes">
<a data-tooltip="Rota da Escola" href="https://www.google.com/maps/dir//-23.0361,-44.9253/@-23.0361,-44.9253,17z/data=!3m1!4b1" id="btnRota" target="_blank">
<img alt="" src="/Contents/img/icones/icon-trajeto.svg"/>
</a> 
                    <a data-tooltip="Informações da Escola" href="/Home/DetalhesEscola?codesc=926085" id="btnDetalhesEscola">
<img alt="" src="/Contents/img/icones/icon-detalhes.svg"/>
</a>
</td>

In [13]:
links = exemplo_ultima_coluna.find_all('a')
link_google = links[0]['href']
link_google

'https://www.google.com/maps/dir//-23.0361,-44.9253/@-23.0361,-44.9253,17z/data=!3m1!4b1'

In [14]:
link_detalhes = links[1]['href']
link_detalhes

'/Home/DetalhesEscola?codesc=926085'

In [15]:
#extrai as coordenadas do link do google maps
def parse_coords_link_google(link_google):
    return link_google.split('/')[-3].split(',')

In [16]:
parse_coords_link_google(link_google)

['-23.0361', '-44.9253']

In [17]:
# extrai o id da escola do link para mais detalhes
def parse_id_escola(link_detalhes):
    return link_detalhes.split('?')[-1].replace('codesc=', '')

In [18]:
parse_id_escola(link_detalhes)

'926085'

In [19]:
#outras colunas sao mais simples, só pegar o texto
def parse_normal_data(td):
    """
    Parses a table cell that contains normal text data.
    """
    return td.text.strip()

In [20]:
[parse_normal_data(td) for td in table.find_all('tr')[1].find_all('td')[:-1]]

['bairro da bocaina', 'guaratingueta', 'cunha']

In [ ]:
# essa classe implementa todo o pipeline e retorna um dataframe
# vou salvar ela como um modulo python para poder reutilizar posteriorment

import requests
from bs4 import BeautifulSoup
import pandas as pd

class PEITableParser:

    URL_POST_ESCOLAS_PEI = 'https://transparencia.educacao.sp.gov.br/Home/conteudoTabelaPrograma'

    PAYLOAD_BUSCA = dict(
    tipo=1,
    #diretoria=,
    #escola=,
    #municipio=,
    #endereco=,
    programa=27 #id do programa PEI
    )

    def get_table_from_pei_website(self) ->str:
        """
        Fetches the PEI data from the website and returns its html.
        """
        with requests.post(self.URL_POST_ESCOLAS_PEI, data=self.PAYLOAD_BUSCA) as response:
            if response.status_code == 200:
                table = response.text
                return table
            else:
                error_msg = f'Falha ao buscar a tabela. Error code: {response.status_code}. Error msg: {response.text}'
                raise RuntimeError(error_msg)

    def get_table_element(self, html_table):
        """
        Parses the HTML table and returns the table element.
        """
        soup = BeautifulSoup(html_table, 'html.parser')
        table = soup.find('table')
        
        if not table:
            raise ValueError("No table found in the HTML content.")

        return table

    def get_cols(self, table) -> list[str]:
        return [th.text.strip() for th in table.find_all('tr')[0].find_all('th')]
    
    
    #extrai as coordenadas do link do google maps
    def parse_coords_link_google(self, link_google)->list[str]:
        return link_google.split('/')[-3].split(',')
    
    # extrai o id da escola do link para mais detalhes
    def parse_id_escola(self, link_detalhes: str) -> str:
        return link_detalhes.split('?')[-1].replace('codesc=', '')
    
    def parse_normal_data(self, td) -> str:
        """
        Parses a table cell that contains normal text data.
        """
        return td.text.strip()
    
    def get_links_from_last_column(self, cells)->list[str]:

        last_col_data = cells[-1]

        link_data = last_col_data.find_all('a') if last_col_data else []
        actual_links = [link['href'] for link in link_data if 'href' in link.attrs]

        return actual_links

    def parse_table(self, table) -> pd.DataFrame:
        """
        Parses the HTML table and returns a DataFrame.
        """
        cols = self.get_cols(table)
        cols.remove('Ações')  # Remove the last column header as it contains links
        cols.append('Coordenadas')
        cols.append('ID Escola')
        data = []

        for row in table.find_all('tr')[1:]:
            cells = row.find_all('td')
            row_data = [self.parse_normal_data(td) for td in cells[:-1]]
            links = self.get_links_from_last_column(cells)
            coords = self.parse_coords_link_google(links[0])
            id_escola = self.parse_id_escola(links[1])
            row_data.append(coords)
            row_data.append(id_escola)

            data.append(row_data)

        return pd.DataFrame(data, columns=cols)
    
    def pipeline(self) -> pd.DataFrame:
        """
        Main pipeline to fetch, parse and return the PEI data as a DataFrame.
        """
        html_table = self.get_table_from_pei_website()
        table = self.get_table_element(html_table)
        df = self.parse_table(table)
        return df
    
    def __call__(self) -> pd.DataFrame:
        """
        Allows the class instance to be called as a function to get the DataFrame.
        """
        return self.pipeline()


In [22]:
parse_pei_table = PEITableParser()

In [23]:
df = parse_pei_table()
df.head()

,Escola,Diretoria de Ensino,Município,Coordenadas,ID Escola
0,bairro da bocaina,guaratingueta,cunha,"[-23.0361, -44.9253]",926085
1,aldeia karugwa,itarare,barao de antonina,"[-23.6028, -49.5874]",268574
2,india maria rosa,penapolis,brauna,"[-21.5656, -50.3173]",395365
3,joao alfredo da silva,presidente prudente,presidente prudente,"[-21.9093, -51.3003]",31756
4,bairro pe da serra,miracatu,iguape,"[-24.3801, -47.5179]",918052


In [24]:
from utils.save_csv import save_csv

save_csv(df, 'pei_escolas_original.csv')

Data saved to data/pei_escolas_original.csv
